In [14]:
import sys

sys.path.insert(0, '/home/jianx/search-exposure/')
import torch
from annoy import AnnoyIndex
import forward_ranker.load_data as load_data
import forward_ranker.train as train
from forward_ranker.utils import print_message
from forward_ranker.test import get_ndcg_precision_rr
obj_reader = load_data.obj_reader
obj_writer = load_data.obj_writer
import pickle
import numpy as np

In [7]:
query_test_emb = obj_reader("/datadrive/ruohan/reverse_ranker/loss_0.09/test_query_new_np.pickle")
qid_test_offset = obj_reader("/datadrive/data/preprocessed_data_with_test/test-query_qid2offset.pickle")
passage_embeddings = obj_reader("/datadrive/ruohan/reverse_ranker/loss_0.09/passage_new_np.pickle")
pid_mapping = obj_reader("/datadrive/jianx/data/annoy/100_ance_passage_map.dict")

print(passage_embeddings.shape)

(8841823, 800)


In [8]:
import faiss
dim = passage_embeddings.shape[1]
cpu_index = faiss.IndexFlatIP(dim)
cpu_index.add(passage_embeddings)

In [9]:
_, dev_I = cpu_index.search(query_test_emb, 1000)

In [10]:
qid_test_mapping = {v: k for k, v in qid_test_offset.items()}
result_dict = {}

for idx, top_list in enumerate(dev_I):
    qid = qid_test_mapping[idx]
    for j, index in enumerate(top_list):
        if index in pid_mapping:
            top_list[j] = pid_mapping[index]
    q_results = {}
    score = float(len(top_list))
    for j, pid in enumerate(top_list):
        q_results[pid] = score
        score -= 1
    result_dict[qid] = q_results
print(result_dict)

{47923: {4297620: 1000.0, 618556: 999.0, 8418681: 998.0, 1681334: 997.0, 6241150: 996.0, 5417582: 995.0, 1681332: 994.0, 6191: 993.0, 6241152: 992.0, 6241151: 991.0, 8641104: 990.0, 473807: 989.0, 6241147: 988.0, 6643066: 987.0, 6241153: 986.0, 5878046: 985.0, 5417578: 984.0, 8641107: 983.0, 388876: 982.0, 8418683: 981.0, 1681331: 980.0, 3306871: 979.0, 5282403: 978.0, 566677: 977.0, 5936354: 976.0, 4652795: 975.0, 3348834: 974.0, 6241154: 973.0, 1681339: 972.0, 3630933: 971.0, 6241148: 970.0, 5417579: 969.0, 8421421: 968.0, 5417581: 967.0, 4436801: 966.0, 8327407: 965.0, 7936847: 964.0, 8641106: 963.0, 1681337: 962.0, 8444865: 961.0, 913044: 960.0, 913042: 959.0, 618554: 958.0, 1659143: 957.0, 8418686: 956.0, 1354798: 955.0, 4297619: 954.0, 7071695: 953.0, 1734759: 952.0, 5417577: 951.0, 4286048: 950.0, 7261339: 949.0, 3008962: 948.0, 1803182: 947.0, 986177: 946.0, 5032362: 945.0, 4793758: 944.0, 5137400: 943.0, 766019: 942.0, 5147526: 941.0, 2333954: 940.0, 476983: 939.0, 7261338: 93

In [15]:
import random
import math
rating_dict = obj_reader("/datadrive/jianx/data/rel_scores.dict")
rank = 10

qids = list(result_dict.keys())
result_ndcg = []
result_prec = []
result_rr = []
for qid in qids:
    if qid in rating_dict:
        ndcg, prec, rr = get_ndcg_precision_rr(rating_dict[qid], result_dict[qid], rank)
        result_ndcg.append(ndcg)
        result_prec.append(prec)
        result_rr.append(rr)
avg_ndcg = np.nanmean(result_ndcg)
avg_prec = np.nanmean(result_prec)
avg_rr = np.nanmean(result_rr)
print(avg_ndcg, avg_prec, avg_rr)

0.4063205057756585 0.46511627906976755 0.6856871406428359


In [44]:
print(qids)

[47923, 1124210, 130510, 405717, 490595, 146187, 1103812, 131843, 1037798, 104861, 915593, 833860, 489204, 264014, 207786, 1121402, 87181, 573724, 962179, 1117099, 182539, 443396, 168216, 148538, 1114819, 156493, 451602, 1121709, 1110199, 359349, 87452, 1133167, 1115776, 183378, 1063750, 1113437, 1112341, 855410, 1106007, 1114646, 1129237, 19335, 527433]


In [1]:
ID = 146187

raw_query_test = {}
with open("/datadrive/jianx/data/msmarco-test2019-queries.tsv") as f:
    for line in f:
        raw_query_test[int(line.split("\t")[0])] = line.split("\t")[1]
print(raw_query_test[ID])

for index in result_dict[ID]:
    print(index, raw_passages[index])

difference between a mcdouble and a double cheeseburger



NameError: name 'result_dict' is not defined

In [1]:
import numpy as np
a = np.array([[11, 12, 13],
                [14, 15, 16],
                [17, 15, 11],
                [12, 14, 15],
              [1,2,3]])
a.reshape(a.shape[0] // 3, -1, a.shape[1])

array([[[11, 12, 13],
        [14, 15, 16],
        [17, 15, 11],
        [12, 14, 15],
        [ 1,  2,  3]]])

In [8]:
all_results = {}
with open("/datadrive/jianx/data/results/all_search_rankings_100_100_flat.csv", "r") as f:
    for line in f:
        qid = int(line.split(",")[0])
        pid = int(line.split(",")[1])
        rank = int(line.split(",")[2])
        if qid not in all_results.keys():
            all_results[qid] = {}
        all_results[qid][pid] = rank

In [16]:
print(len(all_results))

502939


In [15]:
passage_embeddings = obj_reader("/home/jianx/results/passage_0__emb_p__data_obj_0.pb")
print(passage_embeddings.shape)

(8841823, 768)
